# Week 4: Milestone 1 – Smart Tracking System Blockchain Ledger (Draft)

Follow these steps to verify Python↔Ganache↔Smart Contract connection.

1. Setup and Connection

1. **Open Jupyter Notebook**  
2. **Verify Ganache** is running (Desktop App “Quickstart Ethereum” or `ganache-cli`).  
3. **Check RPC port** (default is `7545` in Ganache Desktop).

Below we’ll connect Python → Ganache → our IoTDataStorage contract.

In [8]:
# Cell 1: imports & connection
from web3 import Web3
import json, os

# 1️⃣ Point to your Ganache RPC (update port if needed)
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2️⃣ Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


2. Load Contract ABI and Address

🔗 Now we load the **ABI** exported (in `/artifacts/IoTDataStorageABI.json`) and the **deployed address** from Remix.

In [10]:
# Cell 2: load ABI + contract
abi_path = os.path.join("..", "artifacts", "IoTDataStorageABI.json")
with open(abi_path) as f:
    abi = json.load(f)

# Replace with actual deployed address - convert to checksum format
contract_address_raw = "0xb0f476563ce80ab5d55838542cd6240f662fae01"
contract_address = Web3.to_checksum_address(contract_address_raw)

contract = web3.eth.contract(address=contract_address, abi=abi)

# Use the Ganache account that has a balance
web3.eth.default_account = web3.eth.accounts[1]

print(f"▶️ Using account {web3.eth.default_account}")
print(f"✅ Loaded contract at {contract_address}")

# Optional: Test the connection
try:
    total_records = contract.functions.getTotalRecords().call()
    print(f"📊 Current total records: {total_records}")
    print("🎉 Contract connection successful!")
except Exception as e:
    print(f"❌ Error testing contract connection: {e}")

▶️ Using account 0x4CA114aA8b98547aa5D116b0B164740750807108
✅ Loaded contract at 0xb0f476563CE80aB5d55838542CD6240F662faE01
📊 Current total records: 0
🎉 Contract connection successful!


3. Read-Only Call: getTotalRecords()

Let’s check that our contract is responding:

In [11]:
# Cell 3: read-only call
total = contract.functions.getTotalRecords().call()
print(f"🔢 Total stored entries on‐chain: {total}")

🔢 Total stored entries on‐chain: 0


4. Write a dummy entry

Store one dummy IoT record (TEST001) to prove transactions work.

In [12]:
# Cell 4: store dummy entry

# First, make sure we have a valid account
print("Available accounts:")
for i, account in enumerate(web3.eth.accounts):
    balance = web3.eth.get_balance(account)
    print(f"  [{i}] {account} - Balance: {web3.from_wei(balance, 'ether')} ETH")

# Set the account explicitly (use account[0] which is usually the deployer)
sender_account = web3.eth.accounts[0]  # or web3.eth.accounts[1] if you prefer
print(f"\n▶️ Using account: {sender_account}")

# Store dummy data
tx = contract.functions.storeData(
    "TEST003",      # dummy ID
    "New York",  # data type
    "22.5°C"       # data value
).transact({
    "from": sender_account,  # Use explicit account instead of default_account
    "gas": 200_000,
    "gasPrice": web3.to_wei("1", "gwei")
})

print(f"📝 Transaction sent: {tx.hex()}")

# Wait for transaction to be mined
receipt = web3.eth.wait_for_transaction_receipt(tx)
print(f"✅ Dummy data tx mined: {receipt.transactionHash.hex()}")
print(f"📊 Gas used: {receipt.gasUsed}")

# Verify the data was stored
try:
    total_records = contract.functions.getTotalRecords().call()
    print(f"🎯 Total records after storing: {total_records}")
    
    if total_records > 0:
        # Get the latest record
        latest_record = contract.functions.getRecord(total_records - 1).call()
        print(f"📋 Latest record: {latest_record}")
except Exception as e:
    print(f"❌ Error reading stored data: {e}")

Available accounts:
  [0] 0x00f73505D183C0623937a3DDbC6dCe3Bcf2aA680 - Balance: 99.9989576768125 ETH
  [1] 0x4CA114aA8b98547aa5D116b0B164740750807108 - Balance: 100 ETH
  [2] 0xaC44eEeCe46C51DE9636bA32a41AAdEcB9975A02 - Balance: 100 ETH
  [3] 0x39FbE6bFE9842284C1Afb0700ABfD8c81F98d796 - Balance: 100 ETH
  [4] 0xF15f2B571EEb8C415b1081c6AecFF4Bc343d1bD2 - Balance: 100 ETH
  [5] 0xA215B19Ac952D504b03E8327d4a32c731146F524 - Balance: 100 ETH
  [6] 0xbeb0b281c9D7a43e8cAc4C6168876F45Edb1ff62 - Balance: 100 ETH
  [7] 0xca8e08802aD27E3F7fe2d71d8258f370D824FF23 - Balance: 100 ETH
  [8] 0xC88681516a22B65d14D964e00e26f04c900416e9 - Balance: 100 ETH
  [9] 0xc65CAa3565C3097b777345D66A1fe3CF03DB6aBc - Balance: 100 ETH

▶️ Using account: 0x00f73505D183C0623937a3DDbC6dCe3Bcf2aA680
📝 Transaction sent: bb8e5658a1784b2eb5ca797888f7d86009d6f7f02f54d3cb7c4e455c0c811212
✅ Dummy data tx mined: bb8e5658a1784b2eb5ca797888f7d86009d6f7f02f54d3cb7c4e455c0c811212
📊 Gas used: 144801
🎯 Total records after storing: 1


5. Verify it went through

After mining, call again to see the updated count and fetch the index.

In [17]:
# Cell 5: verify storage
new_total = contract.functions.getTotalRecords().call()
print(f"🔢 New total entries: {new_total}")

first = contract.functions.getRecord(100).call()
print("📦 First stored record:", first)

🔢 New total entries: 101
📦 First stored record: [1748093398, 'SHIP5841', 'FullRow', '2025-05-18 11:53:54|2025-05-18 10:53:54|2025-05-20 11:53:54|2025-05-20 12:58:54|SHIP5841|ORD364752|QC Hub|Cebu City|In Transit|3.74|58.2|False|False|Large Box|True|Heavy Traffic|5.0|True']


In [14]:
# Cell 6: Adding New Iot Data using our Output 1(logistics_data.json) on the Blockchain

import pandas as pd

df = pd.read_csv("../data/logistics_data.csv")

sender_account = web3.eth.accounts[0]
print(f"🔐 Using account: {sender_account}")

for i, row in df.iterrows():
    try:
        # Convert entire row to a single string
        row_string = "|".join(str(row[col]) for col in df.columns)

        tx = contract.functions.storeData(
            str(row['shipment_id']),  # Key or ID
            "FullRow",                # Data type or label
            row_string                # Entire row as single string
        ).transact({
            "from": sender_account,
            "gas": 300_000,
            "gasPrice": web3.to_wei("1", "gwei")
        })

        receipt = web3.eth.wait_for_transaction_receipt(tx)
        print(f"[{i+1}/{len(df)}] ✅ Stored full row for: {row['shipment_id']} | Tx: {tx.hex()}")

    except Exception as e:
        print(f"[{i+1}/{len(df)}] ❌ Error storing full row for {row['shipment_id']}: {e}")


🔐 Using account: 0x00f73505D183C0623937a3DDbC6dCe3Bcf2aA680
[1/100] ✅ Stored full row for: SHIP7734 | Tx: b632774bc9ff4e50d627a935979ec907b0221f38f51ac8b8a1dc87771b668703
[2/100] ✅ Stored full row for: SHIP4309 | Tx: 9143736205aa30d03cb3075dcd26fbee31072e948050a81bdd186b2472e52efb
[3/100] ✅ Stored full row for: SHIP1802 | Tx: 29777255276eebea1e8c8a560ca2c1c18ba62b0b7f7a4bc76a0855284f325908
[4/100] ✅ Stored full row for: SHIP9967 | Tx: 9d9c14035726713a6270e3825b772efade42ff4205b2b96065fa38a0dafbca5b
[5/100] ✅ Stored full row for: SHIP3088 | Tx: 9c0e34e3f018d3d21ed340a77ee5dac3b00f2b8b77bd2c50707c28f3f6d277d3
[6/100] ✅ Stored full row for: SHIP9457 | Tx: 389a272317afd9ec390f246e148ca9eb85abf4e5f475c8a207ec838c3c01efdc
[7/100] ✅ Stored full row for: SHIP7133 | Tx: ca72415e1d467425bb0fb75a7dda97832a8d419815851ae325386be263548ca8
[8/100] ✅ Stored full row for: SHIP5276 | Tx: 58a3ae77316e0247f0060d8dca566f5df2709f9a8cd1b98d15663c10d04b56e7
[9/100] ✅ Stored full row for: SHIP9411 | Tx: 5f722f

In [39]:
# Read first row from CSV

# For this example we use the guide from camu which is the sampler data
first_record = contract.functions.getRecord(0).call()
print("CSV First Record (concatenated):")
print(f"  Package ID: {first_record[0]}")
print(f"  Temperature: {first_record[3]}")

# In this case we use our generated logistics data
# Fetch last record from blockchain
last_record = contract.functions.getRecord(100).call()

print("\nBlockchain Last Record:")
print(f"  Shipment ID: {last_record[0]}")
print(f"  Data Type: {last_record[1]}")
print(f"  Full Row Data: {last_record[2]}")


CSV First Record (concatenated):
  Package ID: 1748092982
  Temperature: 22.5°C

Blockchain Last Record:
  Shipment ID: 1748093398
  Data Type: SHIP5841
  Full Row Data: FullRow
